In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from io import StringIO
from pandas.api.types import is_numeric_dtype
import requests as req
from bs4 import BeautifulSoup
import json
import unidecode

import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn

In [11]:
op = webdriver.ChromeOptions()

op.add_argument('--headless')

driver = webdriver.Chrome('/Applications/chromedriver', options=op)
driver2 = webdriver.Chrome('/Applications/chromedriver', options=op)

In [3]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210707&to=20220501&output=json'
# url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20211231&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [4]:
d = {'page': [], 'section': [], 'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
df = pd.DataFrame(data=d)
df

,page,section,roundup,bulletin_url,bulletin_mps


In [5]:
def getbulurl(link, n):
    driver2.get(link)
    
    ps = driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")
    
    date = [i.text for i in ps if len(i.text) < 18][0]
    reldate = "Released: " + date.rstrip()
    
    print(reldate)
        
    atags = [i for i in driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//a[1]")]
    atags.reverse()
    words = ['Back to the top', 'Data']
    for word in words:
        atags = [i for i in atags if (word not in i.text)]
        
    url1 = [i.get_attribute("href") for i in atags][0]
    
    if url1.split("/")[-1] == 'latest':
        print(url1)
        driver2.get(url1)
        urlPrev = driver2.find_element_by_link_text("View previous releases").get_attribute("href")
        
        url = 'null'
        
        for i in range(1, 20):
            driver2.get(urlPrev + "?page=" + str(i) )
            lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
            listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
            if len(listoas) > 0:
                url = listoas[0].get_attribute("href")
                break
    
        if url == 'null':
                
            urlPrev = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                driver2.get(urlPrev + "?page=" + str(i) )
                lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
                listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
                if len(listoas) > 0:
                    url = listoas[0].get_attribute("href")
                    break
 
    else:
        url = url1
    
    return url

getbulurl('https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25', 3)

Released: 26 January 2021
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/latest


'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/january2021'

In [49]:
def getru(link, df):

    driver.get(link)    
    s = 1
    
    for n in range(3, 16):
        
        for i in range(0, 4):
        
            n1 = n
            n2 = 1+i

            print('n: ', n, 's: ', s, 'n1: ', n1, 'n2: ', n2)

            if driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]"):

                content = ""

                content = content + driver.find_element_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]").text + "\n"

                ps = driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")

                for i, j in enumerate(ps):
                    if (i != 0) & (j.text != '') & (j.text not in ['', 'Back to the top', 'Analysis | Data']):
                        content = content + j.text + "\n"

                if content not in [i for i in df['roundup']]:
                    df = df.append({'page': link, 'section': s, 'roundup': content, 'bulletin_url': getbulurl(link, n), 'bulletin_mps': 'placeholder'}, ignore_index=True)
    
    return df

In [190]:
def testrun(link):
#     driver.get(link)    
    s = 0
    piece = {}
    uninc = ['XLSX', 'Download']
    
    for n in range(3, 16):

        if driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
            for el in driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
                if el.tag_name == 'h3':
                    print(" *** ")
                    s = s+1
                    piece[s] = []
                    piece[s].append(el.text)
                if (el.tag_name == 'p') & (el.text not in ['','🠕 Back to the top']):
                    
                    # Check if contains a link
                    if el.find_elements_by_xpath("a"):
                        for w in uninc:
                            if w in el.find_element_by_xpath("a").text:
                                inc = False
                        if inc:
                            print('el ', el.find_element_by_xpath("a").text)
                        inc = True
                        
                    else:
                        piece[s].append(el.text)
                            
                            
                if (el.tag_name == 'ul') & (el.text not in ['','🠕 Back to the top']):
                    for li in el.find_elements_by_xpath('li'):
                        
                        # Check if contains a link
                        if li.find_elements_by_xpath("a"):
                            for w in uninc:
                                if w in li.find_element_by_xpath("a").text:
                                    inc = False
                            if inc:
                                print('li ', li.find_element_by_xpath("a").text )
                            inc = True
                            
                        else:
                            piece[s].append(" - " + li.text)


                
    return piece

#             if driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n1}]/section/h3[{n2}]'):
#                 elem = driver.find_element_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n1}]/section/h3[{n2}]')
#                 print(elem)
#                 for i in range(10):
#                     print(elem.find_element_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n1}]/section/h3[{n2}]/following-sibling::p | li').text)
# 
#                 s += 1 

pieces = testrun(final_urls[0])
pieces[1]

 *** 
li  Read the latest Coronavirus (COVID-19) Infection Survey bulletin
 *** 
li  Latest bulletin on Coronavirus and the social impacts on Great Britain: 9 July 2021
 *** 
el  daily official government figures
li  Latest bulletin on antibody and vaccination data in the UK
 *** 
el  deaths from all causes in England and Wales in the week ending 25 June 2021
el  local authority and health board
li  Latest bulletin on deaths registered in England and Wales
 *** 
li  Read Deaths registered due to COVID-19: 2020
 *** 
el  93% were positive towards the vaccines in the latest survey
li  Read the latest bulletin on coronavirus and vaccine hesitancy
el  user requested data


['Infections have increased across the UK',
 'Coronavirus (COVID-19) cases increased in England, Wales, Northern Ireland and Scotland in the week ending 3 July 2021.',
 'The estimated percentage of the community population (those not in hospitals, care homes or other institutional settings) that had COVID-19 was:',
 ' - 0.61% in England (1 in 160 people)',
 ' - 0.30% in Wales (1 in 340 people)',
 ' - 0.33% in Northern Ireland (1 in 300 people)',
 ' - 1.01% in Scotland (1 in 100 people)',
 'In England, infections were estimated to be highest in the North East and North West regions.',
 'Across England overall, estimated infection rates increased in all age groups and were highest in those in school Year 12 (aged 16 and 17 years) to age 24 years.']

In [8]:
for url in final_urls:
    df = getru(url, df)

n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7
n 2
n 3
n 4
n 5
n 6
n 7


KeyboardInterrupt: 

In [61]:
final_urls[0]

'https://web.archive.org/web/20210709115515/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights'

In [59]:
driver.get(final_urls[0])

In [39]:
n=3

In [40]:
n1=3

In [41]:
n2=1

In [45]:
n=4
n1=3
n2=2

n=5
n1=3
n2=3

In [54]:
driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n1}]/section/h3[{n2}]')

[]

In [34]:
driver.find_element_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[3]/section/h3[2]').text

'Most report more social distancing than they perceive in others'

In [35]:
driver.find_element_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[4]/section/h3[1]').text

'COVID-19 antibodies continue to rise in line with vaccinations across the UK countries'

In [38]:
driver.find_element_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[5]/section/h3[1]').text

'Deaths in England and Wales fall below five-year average'

In [72]:
driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[3]/section/h3[1]')

[<selenium.webdriver.remote.webelement.WebElement (session="528f8acc10224865415889580d1cb4b6", element="7cec0b36-6ad2-43de-9d8e-f4988155ed87")>]

In [ ]:
'//*[@id="main"]/div[2]/div[1]/article/div[3]/section/h3[1]'